In [1]:
%load_ext sql

In [2]:
# load SQL extension and connect it with the database

import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [3]:
### correct binding issue since SQLAlchemy is version 2
from sqlalchemy import MetaData

%sql sqlite:///my_data1.db

#metadata=MetaData()
#metadata.drop_all(con)
#metadata.create_all(con)
#metadata.reflect(con)

'Connected: @my_data1.db'

In [4]:
import pandas as pd

# create dataframe and transform to SQL
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

In [5]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null;

 * sqlite:///my_data1.db
Done.


[]

In [6]:
### Display the names of all launch sites
%sql select distinct Launch_Site from SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


In [7]:
### Display 5 records with launch sites that start with 'CCA'
%sql select * from SPACEXTABLE where Launch_Site like 'CCA%' limit 5;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [8]:
### Display the total payload mass carried by missions launched by NASA (CRS)
%sql select SUM(PAYLOAD_MASS__KG_) from SPACEXTABLE where Customer like '%CRS%';

 * sqlite:///my_data1.db
Done.


SUM(PAYLOAD_MASS__KG_)
48213


In [9]:
### Display the average payload mass carried by booster version F9 v1.1
%sql select AVG(PAYLOAD_MASS__KG_) from SPACEXTABLE where Booster_Version like '%v1.1%';

 * sqlite:///my_data1.db
Done.


AVG(PAYLOAD_MASS__KG_)
2534.6666666666665


In [10]:
### List the first date when a successful landing was achieved on a ground pad
%sql select MIN(Date) from SPACEXTABLE where Landing_Outcome like '%ground pad%';

 * sqlite:///my_data1.db
Done.


MIN(Date)
2015-12-22


In [11]:
### List the names of boosters that have had successful drone ship landings with 4000kg < payload mass < 6000kg
%sql select distinct Booster_Version from SPACEXTABLE where Landing_Outcome like 'Success (drone ship)' and PAYLOAD_MASS__KG_ > 4000 and PAYLOAD_MASS__KG_ < 6000;

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


In [12]:
### Display the total number of missions with successful landings
%sql select COUNT(*) from SPACEXTABLE where Landing_Outcome like 'Success%';

 * sqlite:///my_data1.db
Done.


COUNT(*)
61


In [36]:
### Display the total number of missions with failed landings
%sql select COUNT(*) from SPACEXTABLE where Landing_Outcome not like 'Success%' and Landing_Outcome not like 'No%';

 * sqlite:///my_data1.db
Done.


COUNT(*)
18


In [15]:
### List the names of boosters that have carried the maximum payload mass
%sql select distinct Booster_Version from SPACEXTABLE where PAYLOAD_MASS__KG_ = (select MAX(PAYLOAD_MASS__KG_) from SPACEXTABLE);

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


In [26]:
### List the records that display month names, failed drone ship landings, boosters, launch site for 2015
%sql select (case SUBSTR(Date,6,2) WHEN '01' THEN 'January' WHEN '02' THEN 'February' WHEN '03' THEN 'March' WHEN '04' THEN 'April' WHEN '05' THEN 'May' WHEN '06' THEN 'June' WHEN '07' THEN 'July' WHEN '08' THEN 'August' WHEN '09' THEN 'September' WHEN '10' THEN 'October' WHEN '11' THEN 'November' WHEN '12' THEN 'December' END) as 'Month', Landing_Outcome, Booster_Version, Launch_Site from SPACEXTABLE where SUBSTR(Date,0,5)='2015' and Landing_Outcome like 'Failure (drone ship)';

 * sqlite:///my_data1.db
Done.


Month,Landing_Outcome,Booster_Version,Launch_Site
January,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
April,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


In [31]:
### Rank in descending order the counts of landing outcomes between 2010-06-04 and 2017-03-20
%sql select Landing_Outcome, count(*) as 'Count' from SPACEXTABLE where Date between '2010-06-04' and '2017-03-20' group by Landing_Outcome order by count(Landing_Outcome) DESC;

 * sqlite:///my_data1.db
Done.


Landing_Outcome,Count
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1
